<a href="https://colab.research.google.com/github/laurence-lin/Kaggle_competition/blob/master/Ashrae_predict_FE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import math

import sklearn
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import os

import gc
from google.colab import files
# load data from Cloud Storage
from google.colab import auth
auth.authenticate_user()

# Configure GCP project and use gsutil to copy the file from storage

!gcloud config set project 'blind-detection'
!gsutil -m cp -r gs://ashare_dataset/*.csv  sample_data/



Updated property [core/project].
Copying gs://ashare_dataset/building_metadata.csv...
Copying gs://ashare_dataset/sample_submission.csv...
Copying gs://ashare_dataset/test.csv...
Copying gs://ashare_dataset/train.csv...
Copying gs://ashare_dataset/weather_test.csv...
Copying gs://ashare_dataset/weather_train.csv...
- [6/6 files][  2.4 GiB/  2.4 GiB] 100% Done  58.6 MiB/s ETA 00:00:00           
Operation completed over 6 objects/2.4 GiB.                                      


In [0]:
# Reduce memory function

# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin
# Modified to support timestamp type, categorical type
# Modified to add option to use float16


from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df


# Load all datasets and reduce memory
print(os.listdir('sample_data/'))
data_path = 'sample_data/'
train = pd.read_csv(os.path.join(data_path, 'train.csv'), parse_dates = ['timestamp'])
test = pd.read_csv(os.path.join(data_path, 'test.csv'), parse_dates = ['timestamp'])
building = pd.read_csv(os.path.join(data_path, 'building_metadata.csv'))
weather_test = pd.read_csv(os.path.join(data_path, 'weather_test.csv'), parse_dates = ['timestamp'])
#submission = pd.read_csv(os.path.join(data_path, 'sample_submission.csv'))
weather_train = pd.read_csv(os.path.join(data_path, 'weather_train.csv'), parse_dates = ['timestamp'])

train = reduce_mem_usage(train, use_float16 = True)
building = reduce_mem_usage(building, use_float16 = True)
weather_train = reduce_mem_usage(weather_train, use_float16 = True)
test = reduce_mem_usage(test)
weather_test = reduce_mem_usage(weather_test)


['anscombe.json', 'README.md', 'weather_train.csv', 'train.csv', 'sample_submission.csv', 'building_metadata.csv', 'test.csv', 'weather_test.csv', 'california_housing_test.csv', 'mnist_train_small.csv', 'california_housing_train.csv', 'mnist_test.csv']
Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 289.19 MB
Decreased by 53.1%
Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.8%
Memory usage of dataframe is 9.60 MB
Memory usage after optimization is: 3.07 MB
Decreased by 68.1%
Memory usage of dataframe is 1272.51 MB
Memory usage after optimization is: 596.49 MB
Decreased by 53.1%
Memory usage of dataframe is 19.04 MB
Memory usage after optimization is: 9.78 MB
Decreased by 48.6%


1. Load dataset 
2. Do EDA to analyze data structure 
3. Do feature engineering 
4. Apply model training 
5. Make test data prediction

In [0]:
# Time alignment

temp_skeleton = pd.concat([weather_train, weather_test], ignore_index = True)
weather_key = ['site_id', 'timestamp']
# Drop samples with same site and same timestamp
temp_skeleton = temp_skeleton[weather_key + ['air_temperature']].drop_duplicates( \
                              subset = weather_key).sort_values(by = weather_key)

# Ranking of temperature in each date, at each site
temp_skeleton['temp_rank'] = temp_skeleton.groupby(['site_id', temp_skeleton['timestamp'].dt.date]) \
                        ['air_temperature'].rank(method = 'average')

# Create a dataframe that consisted of: site_id * hourly mean rank of temperature for 24 hours
df_2d = temp_skeleton.groupby(['site_id', temp_skeleton.timestamp.dt.hour])['temp_rank'].mean().unstack(level = 1)

# Subtract max temperature hourly rank by 14, get time alignment gap: time gap btw 14:00 and peak temp. timing
site_offset = pd.Series(df_2d.values.argmax(axis = 1) - 14)
site_offset.index.name = 'site_id'

def time_align(df):
  # create time offset column
  df['offset'] = df.site_id.map(site_offset)
  df['timestamp_aligned'] = df.timestamp - pd.to_timedelta(df.offset, unit = 'hour')
  df['timestamp'] = df['timestamp_aligned']
  del df['timestamp_aligned']
  return df

# Now, we can align weather_train, weather_test data
weather_train = time_align(weather_train)
weather_test = time_align(weather_test)

del df_2d, temp_skeleton, site_offset

# Do interpolation for weather data first, for too much missing values. There may still be some missing values after this.
# Interpolate by each site across the timestamp
weather_train = weather_train.groupby('site_id').apply(lambda x_site: x_site.interpolate(limit_direction = 'both'))
weather_test = weather_test.groupby('site_id').apply(lambda x_site: x_site.interpolate(limit_direction = 'both'))

print('Missing values in weather train data after interpolation: \n')
print(weather_train.isnull().sum().sort_values(ascending = False))

gc.collect()

Missing values in weather train data after interpolation: 

precip_depth_1_hr     26273
cloud_coverage        17228
sea_level_pressure     8755
offset                    0
wind_speed                0
wind_direction            0
dew_temperature           0
air_temperature           0
timestamp                 0
site_id                   0
dtype: int64


0

I can see that although some NaN values is filled in weather_data, interpolation couldn't fill all of the missing values.  
We will continue this in FE.

In [0]:
# Functions for several preprocessing and feature engineering

# Encode cyclic features
def encode_cyclic_feature(df, col, max_val):
  '''
  Encode cyclic feature with sin cosine transform
  df: dataframe contains cyclic feature
  col: cylcic features to transform
  max_val: max value for that cyclic column
  '''
  df[col + '_sin'] = np.sin(2*np.pi*(df[col]/max_val))
  del df[col]
  return df

# Fill NaNs
def mean_without_overflow_fast(col):
    # Compute mean value of each column which contains missing value
    col /= len(col)
    return col.mean() * len(col)

def fillna(df):
  '''
  Fill NaN for dataframe 
  output: dataframe without missing values
  '''
  # pick up the columns contains null value
  null_col = 100 - df.count()/len(df)*100
  null_col = df.loc[:, null_col > 0] # dataframe from train that contain null columns
  null_col_mean = null_col.apply(mean_without_overflow_fast) # mean value to fill in

  for col in null_col.keys():
    if col == 'year_built' or col == 'floor_count':
      df[col].fillna(math.floor(null_col_mean[col]), inplace = True)
    else:
      df[col].fillna(null_col_mean[col], inplace = True)

  return df

# Create new feature
# Time stamp feature
def time_transform(df):
  df['hour'] = df['timestamp'].dt.hour
  df['year'] = df['timestamp'].dt.year
  df['month'] = df.timestamp.dt.month
  df['day'] = df.timestamp.dt.day
  df['dayofweek'] = df.timestamp.dt.dayofweek
  
  return df

# Create is_holiday feature by US_Holiday calendar
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

# Add is_holiday = 1: The date that within USA_holiday, and the dates that is weekend

date_range = pd.date_range(start = train['timestamp'].min(), end = test['timestamp'].max())
us_holidays = calendar().holidays(start = date_range.min(), end = date_range.max()) # USA holidays within data date_range

gc.collect()

0

In [0]:
print(train.columns)

Index(['building_id', 'meter', 'timestamp', 'meter_reading'], dtype='object')


In [0]:
# Feature engineering
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

def preprocessing(df, building, weather, is_train = False):
  '''
  df: train data or test dataframe
  '''

  # 0. Combine the dataset into one
  df = df.merge(building, on = 'building_id', how = 'left')
  df = df.merge(weather, on = ['site_id', 'timestamp'], how = 'left')

  weather_col = ['air_temperature', 'dew_temperature', 'wind_speed', \
               'wind_direction', 'sea_level_pressure',
               'precip_depth_1_hr', 'cloud_coverage']
  df = df.dropna(subset = weather_col, how = 'all')

  # 1. Create new features
  df = time_transform(df)

  # Only datetime64 could apply isin() function, which is convenient
  df['is_holiday'] = (df['timestamp'].dt.date).astype('datetime64').isin(us_holidays).astype(np.int8)
  df.loc[(df['timestamp'].dt.dayofweek == 5) | (df['timestamp'].dt.dayofweek == 6), 'is_holiday'] = 1

  # 2. Data transformation to make data better for prediction
  # Log transformation for numerical data
  if is_train:
    df['meter_reading'] = np.log1p(df['meter_reading'])
  
  df['square_feet'] = np.log1p(df['square_feet'])

  # Encode cyclic features
  df = encode_cyclic_feature(df, 'dayofweek', 7)
  df = encode_cyclic_feature(df, 'hour', 24)
  df = encode_cyclic_feature(df, 'day', 31)
  df = encode_cyclic_feature(df, 'month', 31)
  
  # 3. Fill NaNs
  df = fillna(df)

  # 4. Categorical encoding
  df['primary_use'] = le.fit_transform(df['primary_use'])

  # 5. Data cleaning: NaN rows, Outliers, ...etc
  drop_features = ['wind_speed', 'sea_level_pressure', 'wind_direction', 'timestamp']
  df.drop(drop_features, axis = 1, inplace = True)

  return df


In [0]:
train = preprocessing(train, building, weather_train, is_train = True)
test = preprocessing(test, building, weather_test, is_train = False)

del building, weather_train, weather_test

In [0]:
print('Train data shape: ', train.shape)
print(train.info())
print('Test data shape: ', test.shape)
print(test.info())

Train data shape:  (20112649, 19)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 20112649 entries, 0 to 20215255
Data columns (total 19 columns):
building_id          int16
meter                int8
meter_reading        float32
site_id              int8
primary_use          int64
square_feet          float64
year_built           float16
floor_count          float16
air_temperature      float16
cloud_coverage       float16
dew_temperature      float16
precip_depth_1_hr    float16
offset               float64
year                 int64
is_holiday           int8
dayofweek_sin        float64
hour_sin             float64
day_sin              float64
month_sin            float64
dtypes: float16(6), float32(1), float64(6), int16(1), int64(2), int8(3)
memory usage: 1.7 GB
None
Test data shape:  (41484919, 19)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 41484919 entries, 0 to 41697599
Data columns (total 19 columns):
row_id               int32
building_id          int16
meter          

In [0]:
train.to_csv('train_processed.csv', index = False)
test.to_csv('test_processed.csv', index = False)